In [1]:
'''

從MySQL取得product table
以及 userID，用來創新文件

'''
import pymysql
import pprint

# open connection(host, user, password, database)
host = "uuboyscymysql.clrhltpp3icl.ap-northeast-1.rds.amazonaws.com"
port = int(3306)
user = "food"
password = "food"
db = "food"

conn = pymysql.connect(
    host = host,
    port = port,
    user = user,
    password = password,
    db = db,
    charset='utf8',    
    cursorclass=pymysql.cursors.DictCursor )

print('Connected successfully!')

cursor = conn.cursor()
cursor.execute("SELECT `productName`, `productUnitPrice` from productTable ORDER BY CAST(productID AS unsigned);")

product_dict_list = cursor.fetchall()

print(product_dict_list)

cursor.execute("SELECT `userID` FROM membersInfo;")

userID_dist_list = cursor.fetchall()
userID_list = []
transaction_status_dict = {}

for i in userID_dist_list:
    userID_list.append(i['userID'])
print(userID_list)

for i in userID_list: # i for userID, j for product name
    transaction_status_dict[i] = {}
    for num, j in enumerate(product_dict_list):
        #print(j['productName'])
        transaction_status_dict[i][j['productName']] = [0, product_dict_list[num]['productUnitPrice'], 0, num+1] # quantity, unit price, subtotal
    
print('\n=====\n')
pprint.pprint(transaction_status_dict)

conn.close()

Connected successfully!
[{'productName': 'Pepper', 'productUnitPrice': 40}, {'productName': 'Egg', 'productUnitPrice': 50}, {'productName': 'Salt', 'productUnitPrice': 30}, {'productName': 'Oil', 'productUnitPrice': 150}, {'productName': 'Soy sauce', 'productUnitPrice': 170}, {'productName': 'Onion', 'productUnitPrice': 75}, {'productName': 'Suger', 'productUnitPrice': 120}, {'productName': 'Cream', 'productUnitPrice': 100}, {'productName': 'Flour', 'productUnitPrice': 50}, {'productName': 'Meat', 'productUnitPrice': 90}]
['U0fe5e2ddf71a138b6e21d4a970592127', 'U19a0dc2519f15c625ea764802cd62202', 'U1e91bf35eb5315bc17da624e46a65c51', 'U26c1d062b5768c0ab8721698973d67b1', 'U2dd03f0598a942d9050eea3c4676f85d', 'U30256c6d3c01f118f8c96f05e1f0f094', 'U3635a6a27c2907e7e6f1cf2d8dd41a0c', 'U400b95f5a7d6cb62ef336f96b26de6dd', 'U57dae482fd0d30eddb584a0e3e69d7fd', 'U5dc989d65f1897bd44fcbd415b47e164', 'U6d22d8292171bb2cfd96d2b9ef5383a9', 'U7101f9f716cfb37a89a2a09207636b17', 'U7173dc299af6ac3fe7697a75c

In [2]:
for i in transaction_status_dict['Ue5148dd83b43580ee1b47aa93a371244']:
    print(i)

Pepper
Egg
Salt
Oil
Soy sauce
Onion
Suger
Cream
Flour
Meat


In [ ]:
from kafka import KafkaConsumer, TopicPartition, KafkaClient
import sys
import json
import requests

secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["35.220.196.201:9092"],
        # ConsumerGroup的名稱
        group_id="chatbot_consumer",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
         key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        value_deserializer=bytes.decode,
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="latest",
    )

####################################################


    # 步驟2.指定想要訂閱訊息的topic名稱
    #topic_name = 'U7173*********************0ad'
    item_num_for_each_userID = {}
    for i in userID_list:
        with open('./transaction_status/%s'%(i), 'w', encoding='utf-8') as f:
            outStr = """
            <tr class="success">
            <td>null</td>
            <td>null</td>
            <td>null</td>
            <td>$ null</td>
            <td>$ null</td>
            </tr>
            """
            f.write(outStr)
        item_num_for_each_userID[i] = 1 # 判斷row顏色、是否刪除第一筆null data
        
########################################
        
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    #consumer.subscribe(topics=topic_name)
    consumer.subscribe(pattern='U[0-9a-z]{32}')
    
    # 表格顏色 list
    tcolor = ["success", "warning"]
    num_color = 0
    #new_transaction = 1
     
    outStr = """"""

    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Start listen incoming messages ...")
        # 持續監控是否有新的record進來
        for record in consumer:
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
            
            wmsg = "topic=%s, partition=%s, offset=%s : (key=%s, value=%s) , timestamp=%s" % (topic, partition,
                                                                              offset, msgKey, msgValue, timestamp)
            # 將消費紀錄寫入檔案
            if not msgKey == 'end':
                if item_num_for_each_userID[topic] == 1:
                    with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                        f.write('')
                
                # 製作dataframe
                outStr = """"""
                col = 0
                total = 0
                for product_status in transaction_status_dict[topic]:
                    if transaction_status_dict[topic][product_status][3] == int(msgKey):
                        transaction_status_dict[topic][product_status][0] = int(msgValue)
                        product_qty = transaction_status_dict[topic][product_status][0]
                        product_unit_price = transaction_status_dict[topic][product_status][1]
                        transaction_status_dict[topic][product_status][2] = product_qty * product_unit_price
                        subtotal = transaction_status_dict[topic][product_status][2]
                    else:
                        product_qty = transaction_status_dict[topic][product_status][0]
                        product_unit_price = transaction_status_dict[topic][product_status][1]
                        subtotal = transaction_status_dict[topic][product_status][2]
                    total += subtotal
                    
                    if not product_qty == 0:
                        outStr += """
                            <tr class=%s>
                            <td>%s</td>
                            <td>%s</td>
                            <td>%s</td>
                            <td>$ %s</td>
                            <td>$ %s</td>
                            </tr>
                        """%(tcolor[col%2], col+1,
                                product_status, 
                                product_qty,
                                product_unit_price,
                                subtotal
                            )
                        col += 1
                outStr += """
                    <thead>
                        <tr class="danger">
                        <th>Total</th>
                        <th>-</th>
                        <th>-</th>
                        <th>-</th>
                        <th>$ %s</th>
                        </tr>
                    </thead>
                    """%(total)
                with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                    f.write(outStr)
                
                '''with open('./transaction_status/%s'%(topic), 'a', encoding='utf-8') as f:
                    outStr = """
                        <tr class=%s>
                        <td>%s</td>
                        <td>%s</td>
                        <td>%s</td>
                        <td>$ %s</td>
                        <td>$ null</td>
                        </tr>
                    """%(tcolor[item_num_for_each_userID[topic]%2], item_num_for_each_userID[topic], 
                                                                    product_dict_list[int(msgKey)-1]['productName'],
                                                                    str(msgValue),
                                                                    product_dict_list[int(msgKey)-1]['productUnitPrice']
                        )
                    f.write(outStr)'''
                item_num_for_each_userID[topic] += 1
                
            if msgKey == 'end':
                with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                    outStr = """
                        <tr class="success">
                        <td>null</td>
                        <td>null</td>
                        <td>null</td>
                        <td>$ null</td>
                        <td>$ null</td>
                        </tr>
                    """
                    f.write(outStr)
                # 將該次交易紀錄上傳MySQL
                conn = pymysql.connect(
                        host = host,
                        port = port,
                        user = user,
                        password = password,
                        db = db,
                        charset='utf8',    
                        cursorclass=pymysql.cursors.DictCursor )
                cursor = conn.cursor()
                
                for product_status in transaction_status_dict[topic]:
                    if not transaction_status_dict[topic][product_status][0] == 0:
                        cursor.execute("INSERT INTO transaction SET orderID=%s, productID=%s, userID=%s, productQTY=%s, datetime=FROM_UNIXTIME(%s)",
                                          (int(timestamp/1000),
                                          transaction_status_dict[topic][product_status][3],
                                          topic,
                                          transaction_status_dict[topic][product_status][0],
                                          int(timestamp/1000)
                                          )
                                      )
                        '''sql += "(%s,%s,'%s',%s,FROM_UNIXTIME(%s));"%(
                                                                    int(timestamp/1000),
                                                                    msgKey,
                                                                    topic,
                                                                    transaction_status_dict[topic][product_status][0],
                                                                    int(timestamp/1000)
                                                                    )
                        '''
                        #print('-----%s'%(sql))
                conn.commit()
                conn.close()
                requests.get('https://%s/transaction_commit/%s'%(server_url, topic))
                print('\nFrom : %s\nTransaction Completed!\n'%(topic))
                
                item_num_for_each_userID[topic] = 1
                
            # 秀出metadata與msgKey & msgValue訊息
            print(wmsg)
            
    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        print('Bye!')
        conn.close()
        consumer.close()

Start listen incoming messages ...
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=16 : (key=1, value=0) , timestamp=1552184896954
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=17 : (key=2, value=0) , timestamp=1552184896954
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=18 : (key=3, value=0) , timestamp=1552184896954
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=19 : (key=4, value=0) , timestamp=1552184896954
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=20 : (key=5, value=0) , timestamp=1552184896954
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=21 : (key=6, value=0) , timestamp=1552184896955
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=22 : (key=7, value=0) , timestamp=1552184896955
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=23 : (key=8, value=0) , timestamp=1552184896955
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=24 : (key=9, value=0) , timestam

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=26 : (key=6, value=3) , timestamp=1552184916964
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=26 : (key=2, value=1) , timestamp=1552184922964
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=26 : (key=2, value=2) , timestamp=1552184927965
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=25 : (key=7, value=0) , timestamp=1552184935966
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=26 : (key=2, value=2) , timestamp=1552184941967
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=25 : (key=6, value=1) , timestamp=1552184945968
topic=U30256c6d3c01f118f8c96f05e1f0f094, partition=0, offset=25 : (key=4, value=3) , timestamp=1552184950969
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=25 : (key=6, value=1) , timestamp=1552184955970
topic=U26c1d062b5768c0ab8721698973d67b1, partition=0, offset=25 : (key=1, value=0) , timestamp=1552184961971
topic=U26c1d062b576

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=39 : (key=1, value=3) , timestamp=1552217605049
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=39 : (key=8, value=3) , timestamp=1552217610050
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=39 : (key=5, value=2) , timestamp=1552217616051
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=39 : (key=7, value=3) , timestamp=1552217624052
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=39 : (key=8, value=1) , timestamp=1552217632053
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=38 : (key=5, value=1) , timestamp=1552217637054
topic=U30256c6d3c01f118f8c96f05e1f0f094, partition=0, offset=38 : (key=8, value=0) , timestamp=1552217643055
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=38 : (key=2, value=0) , timestamp=1552217648055
topic=U26c1d062b5768c0ab8721698973d67b1, partition=0, offset=39 : (key=2, value=2) , timestamp=1552217653057
topic=U7173dc299af6

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=53 : (key=10, value=0) , timestamp=1552247839107
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=43 : (key=1, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=44 : (key=2, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=45 : (key=3, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=46 : (key=4, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=47 : (key=5, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=48 : (key=6, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=49 : (key=7, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=50 : (key=8, value=0) , timestamp=1552247839110
topic=Ua0ac6b70d36

topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=59 : (key=5, value=2) , timestamp=1552248183149
topic=U30256c6d3c01f118f8c96f05e1f0f094, partition=0, offset=58 : (key=2, value=1) , timestamp=1552248188149
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=59 : (key=4, value=3) , timestamp=1552248193149
topic=U26c1d062b5768c0ab8721698973d67b1, partition=0, offset=60 : (key=10, value=0) , timestamp=1552248198150
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=60 : (key=4, value=1) , timestamp=1552248204150
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=63 : (key=6, value=2) , timestamp=1552248208151
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=61 : (key=10, value=0) , timestamp=1552248213152
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=61 : (key=5, value=2) , timestamp=1552248220152
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=60 : (key=4, value=3) , timestamp=1552248225153
topic=U9a1715c6ff

...

topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=439 : (key=8, value=0) , timestamp=1552573587046
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=440 : (key=9, value=0) , timestamp=1552573587046
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=441 : (key=10, value=0) , timestamp=1552573587046
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=444 : (key=10, value=1) , timestamp=1552573607050
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=589 : (key=6, value=0) , timestamp=1552573611051
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=447 : (key=4, value=0) , timestamp=1552573617052
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=442 : (key=10, value=2) , timestamp=1552573623052
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=441 : (key=3, value=0) , timestamp=1552573629053
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=446 : (key=8, value=2) , timestamp=1552573635054
topic=U

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=456 : (key=5, value=2) , timestamp=1552578968821
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=601 : (key=5, value=3) , timestamp=1552578972822
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=459 : (key=1, value=1) , timestamp=1552578979823
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=454 : (key=6, value=1) , timestamp=1552578983824
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=454 : (key=5, value=1) , timestamp=1552578991825
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=458 : (key=7, value=3) , timestamp=1552578998827
topic=U30256c6d3c01f118f8c96f05e1f0f094, partition=0, offset=456 : (key=3, value=2) , timestamp=1552579003828
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=456 : (key=2, value=0) , timestamp=1552579009829
topic=U26c1d062b5768c0ab8721698973d67b1, partition=0, offset=458 : (key=8, value=2) , timestamp=1552579014831
topic=Ua0a

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=610 : (key=1, value=0) , timestamp=1552599043071
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=611 : (key=2, value=0) , timestamp=1552599043071
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=612 : (key=3, value=0) , timestamp=1552599043072
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=613 : (key=4, value=0) , timestamp=1552599043072
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=464 : (key=1, value=0) , timestamp=1552599043069
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=465 : (key=2, value=0) , timestamp=1552599043069
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=466 : (key=3, value=0) , timestamp=1552599043069
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=467 : (key=4, value=0) , timestamp=1552599043069
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=468 : (key=5, value=0) , timestamp=1552599043069
topic=U717

topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=463 : (key=1, value=0) , timestamp=1552599043078
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=464 : (key=2, value=0) , timestamp=1552599043078
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=465 : (key=3, value=0) , timestamp=1552599043079
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=466 : (key=4, value=0) , timestamp=1552599043079
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=467 : (key=5, value=0) , timestamp=1552599043079
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=468 : (key=6, value=0) , timestamp=1552599043079
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=469 : (key=7, value=0) , timestamp=1552599043079
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=470 : (key=8, value=0) , timestamp=1552599043079
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=471 : (key=9, value=0) , timestamp=1552599043079
topic=U934

topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=484 : (key=8, value=0) , timestamp=1552633191117
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=485 : (key=9, value=0) , timestamp=1552633191118
topic=U934211376c8da4cc677c3b801bd9f8db, partition=0, offset=486 : (key=10, value=0) , timestamp=1552633191118
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=487 : (key=1, value=1) , timestamp=1552633211118
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=633 : (key=8, value=0) , timestamp=1552633215118
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=489 : (key=4, value=3) , timestamp=1552633221118
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=484 : (key=4, value=2) , timestamp=1552633227118
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=485 : (key=5, value=0) , timestamp=1552633235119
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=490 : (key=7, value=3) , timestamp=1552633243120
topic=U30